In [213]:
import random
import math
import statistics
import matplotlib.pyplot as plt
import numpy as np
from pprint import pprint
from deap import base, creator, tools, algorithms

In [214]:
random.seed(123456)

## Iterované vězňovo dilema

In [215]:
def scores(move1, move2):
    # 1 = betray, 0 = not betray

    scores = (0, 0)

    if (move1 == 1) and (move2 == 1):
        scores = (2, 2)

    if (move1 == 1) and (move2 == 0):
        scores = (0, 3)

    if (move1 == 0) and (move2 == 1):
        scores = (3, 0)

    if (move1 == 0) and (move2 == 0):
        scores = (1, 1)

    return scores

def play(player1_strategy, player2_strategy, num_steps, pprint=False):

    score1 = 0
    score2 = 0

    history1 = []
    history2 = []

    for i in range(num_steps):
        move1 = player1_strategy(history1, history2)
        move2 = player2_strategy(history2, history1)

        s1, s2 = scores(move1, move2)
        score1 += s1
        score2 += s2

        history1.append(move1)
        history2.append(move2)
    if pprint: print("Historie hráče 1: ", history1)
    if pprint: print("Historie hráče 2: ", history2)
    if pprint: print("Počet let za mřížemi: ", score1, score2)
    return score1, score2


## Special cases

#### initial_betrayal
betraying when playing the first round (no history)

#### cautious_vs_sorry
betraying after successfully getting away with it (3, 0) case

#### insidious_vs_peaceful
betraying after the opponent hasn't betrayed me twice

#### revenge_vs_chance
betraying after being betrayed three times in a row

## Cases based on score
winning significantly, loosing significantly or the average case

In [216]:
def strategy(initial_betrayal, cautious_vs_sorry, insidious_vs_peaceful, revenge_vs_chance, winning, loosing, average):

    def configured_strategy(my_history, opponents_history):
        my_move = 0
        if len(my_history) == 0:
            my_move = initial_betrayal
        elif my_history[-1] == 1 and opponents_history[-1] == 0:
            my_move = cautious_vs_sorry
        elif opponents_history[-2:] == [0, 0]:
            my_move = insidious_vs_peaceful
        elif opponents_history[-3:] == [1, 1, 1]:
            my_move = revenge_vs_chance
        else:
            my_score, opponents_score = tuple(sum(pair) for pair in zip(*[scores(x, y) for x, y in zip(my_history, opponents_history)]))
            if my_score / (opponents_score + 0.00001) > 1.25:
                my_move = winning
            elif my_score / (opponents_score + 0.00001) < 0.75:
                my_move = loosing
            else:
                my_move = average      
        return my_move

    return configured_strategy
    

In [217]:
def fitness(my_strategy, population):
    my_strategy = strategy(*my_strategy)
    my_score = 0
    for other_individual in population:
        opponent_strategy = strategy(*other_individual)
        my_score_iter, opponent_score = play(my_strategy, opponent_strategy, 10)
        my_score += my_score_iter
    return (my_score,)
    

In [218]:
creator.create("FitnessMax", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)
toolbox = base.Toolbox()
toolbox.register("attr_int", random.randint, 0, 1)
toolbox.register("individual", tools.initRepeat, creator.Individual,
                 toolbox.attr_int, n=7)
toolbox.register("population", tools.initRepeat, list, toolbox.individual) 

toolbox.register("evaluate", lambda individual: fitness(individual, population))

toolbox.register("mate", tools.cxOnePoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)

population = toolbox.population(n=20)
NGEN = 500            
CXPB = 0.7           
MUTPB = 0.2       

finalpopulation, logbook = algorithms.eaSimple(population, toolbox, cxpb=CXPB, mutpb=MUTPB, ngen=NGEN)

gen	nevals
0  	20    
1  	17    
2  	11    
3  	13    
4  	10    
5  	20    
6  	18    
7  	16    
8  	15    
9  	10    
10 	17    
11 	16    
12 	13    
13 	19    
14 	16    
15 	17    
16 	16    
17 	12    
18 	18    
19 	11    
20 	16    
21 	18    
22 	13    
23 	10    
24 	14    
25 	14    
26 	16    
27 	15    
28 	17    
29 	14    
30 	16    
31 	15    
32 	15    
33 	15    
34 	16    
35 	18    
36 	16    
37 	10    
38 	16    
39 	15    
40 	20    
41 	16    
42 	17    
43 	17    
44 	16    
45 	15    
46 	16    
47 	16    
48 	15    
49 	13    
50 	13    
51 	16    
52 	16    
53 	10    
54 	15    
55 	10    
56 	17    
57 	17    
58 	14    
59 	15    
60 	13    
61 	18    
62 	11    
63 	16    
64 	13    
65 	18    
66 	18    
67 	15    
68 	16    
69 	18    
70 	18    
71 	15    
72 	16    
73 	15    
74 	18    
75 	15    
76 	11    
77 	18    
78 	16    
79 	14    
80 	12    
81 	13    
82 	12    
83 	16    
84 	14    
85 	14    
86 	13    
87 	17    
88 	9     
89 	18    

### souboj evolvovaných jedinců

In [219]:
play(strategy(*finalpopulation[0]), strategy(*finalpopulation[1]), 10, True)

Historie hráče 1:  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Historie hráče 2:  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Počet let za mřížemi:  10 10


(10, 10)

In [220]:
for i in finalpopulation:
    print(i)

[0, 0, 0, 1, 1, 0, 0]
[0, 0, 0, 1, 1, 0, 0]
[0, 0, 0, 1, 1, 0, 0]
[0, 1, 0, 0, 1, 0, 0]
[0, 0, 0, 1, 1, 0, 0]
[0, 0, 0, 0, 1, 0, 0]
[0, 0, 0, 1, 1, 0, 0]
[0, 0, 0, 1, 1, 0, 0]
[0, 0, 0, 1, 1, 0, 0]
[0, 0, 0, 1, 1, 0, 0]
[0, 0, 0, 1, 1, 0, 0]
[0, 0, 0, 1, 1, 0, 0]
[0, 1, 0, 0, 1, 0, 0]
[0, 0, 0, 1, 1, 0, 0]
[0, 0, 0, 1, 1, 0, 0]
[0, 0, 0, 1, 1, 0, 0]
[0, 0, 0, 1, 1, 0, 0]
[0, 0, 0, 1, 1, 0, 0]
[0, 0, 0, 1, 1, 0, 0]
[0, 0, 0, 1, 1, 0, 0]


Jako optimální se jeví následující taktika:
- v prvním kole nezradit
- po úspěšném zrazení (a vyváznutí bez trestu) nezradit podruhé
- v případě, že mě soupeř dvakrát za sebou nezradil ho také nezradím
- **pokud jsem třikrát za sebou zrazen, pomstím se**
- **pokud vyhrávám (slabý soupeř?) zrazuji**
- jinak nezrazuji

Pokud taktiku aplikují všichni v gangu vede to k tomu, že se vzájemně nezrazují a celkově vyváznou s minimálním možným trestem (z pohledu celé skupiny)